<a href="https://colab.research.google.com/github/DaeSeokSong/LSTM-PPoA/blob/main/Predict_Price_of_Agricultural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 참조문헌
## [A Prediction Model for Agricultural Products Price with LSTM Network](https://www.koreascience.or.kr/article/JAKO201809469053682.page)

**신성호, 이미경, 송사광**

한국과학기술정보연구원 연구데이터플랫폼센터,
한국과학기술정보연구원 연구데이터플랫폼센터/과학기술연합대학원대학교 빅데이터과학과

Sungho Shin(maximus74@kisti.re.kr), Mikyoung Lee(jerryis@kisti.re.kr),
Sa-kwang Song(esmallj@kisti.re.kr)

# 당년 농산물 가격 예측 LSTM 모델

Input = 1~저번 달까지의 pram 값을 하나로 묶은 array(인스턴스)

output = 당월의 해당 채소 가격

layer = 원래 해당 채소 가격

### Google Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/DeepLearning/Project/PPoA
!ls -al

Mounted at /content/gdrive
/content/gdrive/MyDrive/DeepLearning/Project/PPoA
total 12
drwx------ 2 root root 4096 Nov 24 06:07  Dataset
-rw------- 1 root root 7547 Nov 25 14:00 'Predict Price of Agricultural.ipynb'


### Import


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.utils import *
from sklearn.preprocessing import *

### Function

In [3]:
def Normalizer(targetData) :
    return (targetData - targetData.min()) / (targetData.max() - targetData.min())

### Global variable

In [4]:
TARGET_YEAR = 2020
START_YEAR = 0
END_YEAR = 0
CROPS = ''

### Data read

In [77]:
START_YEAR = 2006
END_YEAR = 2019
CROPS = 'onion'
dataset = []

# 지난 최대 5년간 최대, 최소치를 기준으로 정규화해야 하는 항목들
df = pd.read_csv('./Dataset/last_production_'+CROPS+'.csv', index_col=0, encoding='cp949')
# 전년 재배면적
last_cultiv_area = df['전년면적']
# 전년 평균 생산량(수확량)
last_production = df['전년생산량']
# 전년 평균 생산단수
last_prod_unit = df['전년단수']
last_prams = [last_cultiv_area,                    # 전년 재배면적
            last_production,                       # 전년 평균 생산량
            last_prod_unit                         # 전년 평균 생산단수
            ]

for next in range(0, (END_YEAR-START_YEAR)+1) :
    cur_year = START_YEAR + next

    f_name = str(cur_year)+'_'+CROPS+'.csv'
    df = pd.read_csv('./Dataset/'+f_name, index_col=0)

    ''' 기상변수 '''
    # 강수량
    precipi_avg = df['평균월강수량(mm)']
    precipi_max = df['최다월강수량(mm)']
    # 기온
    temper_avg = df['평균기온(℃)']
    temper_max = df['평균최고기온(℃)']
    temper_min = df['평균최저기온(℃)']
    # 풍속
    windSpeed_avg = df['평균풍속(m/s)']
    windSpeed_max = df['최대풍속(m/s)']
    # 습도
    humidity_avg = df['평균습도(%rh)']
    humidity_min = df['최저습도(%rh)']
    # 일조량 / 일사량
    sunshine = df['일조합']
    insolation = df['일사합']

    ''' 기타변수 '''
    # 전년 수입량
    last_amount_import = df['전년수입량']
    # 해당 농작물 가격 (index == 12)
    crops_price = df['가격']
    # 경유 가격
    diesel_price = df['경유가격']
    # 물가지수(price index -> pidx), 2015년 기준 얼마나 오르고 내렸는지
    total_pidx = df['총물가지수']
    prod_pidx = df['상품']
    agricul_marine_prod_pidx = df['농축수산물']
    indust_prod_pidx = df['공업제품']
    serv_pidx = df['서비스']
    pub_serv_pidx = df['공공서비스']
    per_serv_pidx = df['개인서비스']
    house_pidx = df['집세']

    # 정규화 해야하는 데이터셋
    prams = [precipi_avg, precipi_max,             # 강수량
            temper_avg, temper_max, temper_min,    # 기온
            windSpeed_avg, windSpeed_max,          # 풍속
            humidity_avg, humidity_min,            # 습도
            sunshine,                              # 일조량
            insolation,                            # 일사량
            crops_price,                           # 해당 농작물 월별 가격
            diesel_price,                          # 월별 경유 가격
            last_amount_import,                    # 전년 수입량
            # 물가지수
            total_pidx, prod_pidx, agricul_marine_prod_pidx, indust_prod_pidx, serv_pidx, pub_serv_pidx, per_serv_pidx, house_pidx,
            ]

    dataset.append(prams)

### Data preprocessing

In [78]:
''' 정규화 '''
# last_prams
norm_last = []
for idx, pram in enumerate(last_prams) :
    tmp_last = []
    for next in range(0, (END_YEAR-START_YEAR)+1) :
        tmp_df = pram

        cur_year = START_YEAR + next
        year_list = tmp_df.index.values.tolist()
        if cur_year-4 < year_list[-1] : year_list = year_list[year_list.index(cur_year):len(year_list)] # 5년간 데이터가 없을 경우
        else : year_list = year_list[year_list.index(cur_year):year_list.index(cur_year-4)+1]           # 5년간 데이터가 있는 경우

        tmp_df = tmp_df.loc[year_list]
        tmp_last.append(Normalizer(tmp_df).to_numpy()[0])
    norm_last.append(tmp_last)
# prams
for idx, data in enumerate(dataset) :
    for i, pram in enumerate(data) :
        pram = Normalizer(pram)
        pram = pram.to_numpy()

        dataset[idx][i] = pram

''' 데이터셋 구축 '''
X_dataset = []
y_dataset = []

# X, y 데이터셋 구분
for year in dataset :
    tmp_X1 = [[], [], [], [], []] # 1~5월
    tmp_y1 = [] # 6월
    tmp_X2 = [[], [], [], [] ,[]] # 7~11월
    tmp_y2 = [] # 12월
    for idx, data in enumerate(year) :
        for month in range(0, 12) :
            if idx == 12 : 
                if month == 5 : tmp_y1.append(data[month])
                elif month == 11: tmp_y2.append(data[month])
            else :
                if month < 5 : tmp_X1[month].append(data[month])
                elif month > 5 and month < 11 : tmp_X2[month-6].append(data[month])

    X_dataset.append(tmp_X1)
    y_dataset.append(tmp_y1)
    X_dataset.append(tmp_X2)
    y_dataset.append(tmp_y2)

In [79]:
print(X_dataset)
print(y_dataset)

[[[0.02398081534772182, 0.028084167656541658, 0.0, 0.0, 0.0, 0.3333333333333332, 0.2978723404255319, 0.2903225806451613, 0.125, 0.5798045602605862, 0.062057240011334676, 0.10435931307793923, 0.1579894973743436, 0.0, 0.1671217913708339, 0.5365893233814707, 0.0, 0.0, 0.0, 0.0, 0.0], [0.02749800159872103, 0.03122348549126082, 0.023166023166023165, 0.03088803088803088, 0.003773584905660364, 0.7499999999999998, 1.0, 0.1935483870967742, 0.125, 0.6436482084690553, 0.2621768835993829, 0.18494055482166447, 0.030307576894223556, 0.0915411355735813, 0.1671217913708339, 0.4634106766185293, 0.05832693783575869, 0.12012012012011813, 0.10003862495171406, 0.15361744301288513, 0.07677356656949272], [0.0, 0.0, 0.20463320463320467, 0.2509652509652509, 0.14339622641509434, 0.9166666666666666, 0.6595744680851066, 0.0, 0.0, 1.0, 0.7857750070841598, 0.5204755614266843, 0.2123780945236309, 0.36384704519119376, 0.1671217913708339, 0.4634106766185293, 0.05832693783575869, 0.4799799799799839, 0.2000772499034336,

### Modeling

In [ ]:
model = Sequential()

model.add(LSTM(
    1, # 해당 층의 노드 개수
    input_shape=(50,1), # input_shape=?
    return_sequences=True)) # return_sequences == 각 시퀀스를 출력할지
model.add(Dropout(0.01)) # 과적합 방지용 Dropout 20%(==0.2)

model.add(Dense(1, activation='sigmoid')) # 활성화(Activation)함수 = sigmoid
model.compile(loss='mse', optimizer='rmsprop')

### Learning

### Prediction